In [1]:
config_file = 'config_run3.json'

In [2]:
import os
import json
import argparse
import glob
import pandas as pd
import numpy as np

from pinf.analytical_field import get_analytic_b_field
from pinf.unpack import load_cube
from pinf.potential_field import get_potential
from pinf.plot import pv_plot
from pinf.performance_metrics import metrics 

with open(config_file) as config:
    info = json.load(config)

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= info['simul']['gpu_id']

def mag_plot(title, mag, vtk_path):
    vtk_file = os.path.join(vtk_path, f'{title}.vtk')
    p = pv_plot(B=mag, vtk_path=vtk_file, points=((16, 49, 8), (16, 49, 8)))
    p.camera_position = 'xy'
    p.show(jupyter_backend='static')

    p.camera_position = 'yz'
    p.show(jupyter_backend='static')

    p.camera_position = 'xz'
    p.show(jupyter_backend='static')

    p.camera_position = 'xz'
    p.camera.azimuth = -30
    p.camera.elevation = 25
    p.show(jupyter_backend='static')

def metric_df(B, b, B_potential, iteration):
    metric = metrics(B=B, b=b, B_potential=B_potential)
    iterinfo = {'iter': iteration}
    metric = {**iterinfo, **metric}

    df = pd.DataFrame.from_dict([metric])
    return df

vtk_path = info['output']['vtk_path']
metric_path = info['output']['metric_path']
plot_path = info['output']['plot_path']
os.makedirs(vtk_path, exist_ok=True)
os.makedirs(metric_path, exist_ok=True)
os.makedirs(plot_path, exist_ok=True)

n = info['exact']['n']
m = info['exact']['m']
l = info['exact']['l']
psi = eval(info['exact']['psi'])
resolution = info['exact']['resolution']
bounds = info['exact']['bounds']

In [3]:
b = get_analytic_b_field(n=n, m=m, l=l, psi=psi, resolution=resolution, bounds=bounds)

potential = get_potential(b[:, :, 0, 2], b.shape[2], batch_size=1000)
b_potential = - 1 * np.stack(np.gradient(potential, axis=[0, 1, 2], edge_order=2), axis=-1)

Potential Field: 100%|██████████| 263/263 [00:01<00:00, 212.44it/s]


In [4]:
from pinf.metric import vector_norm, curl, divergence

In [5]:
j_b = curl(b)
vector_norm(j_b).sum()

104301.65490588089

In [6]:
vector_norm(np.cross(j_b, b, -1)).sum()

94244.36856992802

In [7]:
vector_norm(b).sum()

1131335.6931783683

In [8]:
(vector_norm(np.cross(j_b, b, -1)) / vector_norm(b)).mean()

0.005204202540954063

In [9]:
(vector_norm(np.cross(j_b, b, -1)) / vector_norm(b)).sum()

1364.2504708958618

In [10]:
(vector_norm(np.cross(j_b, b, -1)) / vector_norm(b)).sum() / vector_norm(j_b).sum()

0.01307985450592252

In [11]:
j_pot = curl(b_potential)
vector_norm(j_pot).sum()

0.028810939

In [12]:
vector_norm(np.cross(j_pot, b_potential, -1)).sum()

0.95010036

In [13]:
vector_norm(b_potential).sum()

748053.9

In [14]:
(vector_norm(np.cross(j_pot, b_potential, -1)) / vector_norm(b_potential)).mean()

9.375654e-08

In [15]:
(vector_norm(np.cross(j_pot, b_potential, -1)) / vector_norm(b_potential)).mean()

9.375654e-08

In [16]:
(vector_norm(np.cross(j_pot, b_potential, -1)) / vector_norm(b_potential)).sum() / vector_norm(j_pot).sum()

0.8530688

In [17]:
import os
import json
import argparse
import glob
import pandas as pd
import numpy as np

from pinf.analytical_field import get_analytic_b_field
from pinf.unpack import load_cube
from pinf.potential_field import get_potential
from pinf.plot import pv_plot
from pinf.performance_metrics import metrics 

with open(config_file) as config:
    info = json.load(config)

os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]= info['simul']['gpu_id']

def mag_plot(title, mag, vtk_path, plot_path):
    vtk_file = os.path.join(vtk_path, f'{title}.vtk')
    p = pv_plot(B=mag, vtk_path=vtk_file, points=((16, 49, 8), (16, 49, 8)))

    xy_path = os.path.join(plot_path, f'{title}_xy.pdf')
    yz_path = os.path.join(plot_path, f'{title}_yz.pdf')
    xz_path = os.path.join(plot_path, f'{title}_xz.pdf')
    xz_tilted_path = os.path.join(plot_path, f'{title}_xz_tilted.pdf')

    if not os.path.exists(xy_path):
        p.camera_position = 'xy'
        p.save_graphic(xy_path)

    if not os.path.exists(yz_path):
        p.camera_position = 'yz'
        p.save_graphic(yz_path)

    if not os.path.exists(xz_path):
        p.camera_position = 'xz'
        p.save_graphic(xz_path)  

    if not os.path.exists(xz_tilted_path):
        p.camera_position = 'xz'
        p.camera.azimuth = -30
        p.camera.elevation = 25
        p.save_graphic(xz_tilted_path)

def metric_df(B, b, B_potential, iteration):
    metric = metrics(B=B, b=b, B_potential=B_potential)
    iterinfo = {'iter': iteration}
    metric = {**iterinfo, **metric}

    df = pd.DataFrame.from_dict([metric])
    return df

vtk_path = info['output']['vtk_path']
metric_path = info['output']['metric_path']
plot_path = info['output']['plot_path']
os.makedirs(vtk_path, exist_ok=True)
os.makedirs(metric_path, exist_ok=True)
os.makedirs(plot_path, exist_ok=True)

n = info['exact']['n']
m = info['exact']['m']
l = info['exact']['l']
psi = eval(info['exact']['psi'])
resolution = info['exact']['resolution']
bounds = info['exact']['bounds']

b = get_analytic_b_field(n=n, m=m, l=l, psi=psi, resolution=resolution, bounds=bounds)

potential = get_potential(b[:, :, 0, 2], b.shape[2], batch_size=1000)
b_potential = - 1 * np.stack(np.gradient(potential, axis=[0, 1, 2], edge_order=2), axis=-1)

df_b = metric_df(B=b, b=b, B_potential=b_potential, iteration=-2)

df_pot = metric_df(B=b_potential, b=b, B_potential=b_potential, iteration=-1)
df = pd.concat([df_b, df_pot], ignore_index=True)

field_files = os.path.join(info['simul']['base_path'],'fields_*.nf2')
field_files = sorted(glob.glob(field_files))

file_path = field_files[0]
iters = os.path.basename(file_path).split('.')[0][7:]
title = 'PINN' + '_' + iters
B = load_cube(file_path)

import torch 

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
loss_path = file_path.replace('fields_', 'loss_')
state = torch.load(loss_path, map_location=device)

Potential Field: 100%|██████████| 263/263 [00:00<00:00, 266.82it/s]


In [18]:
state

{'BC_loss': array(0.0402357, dtype=float32),
 'lambda_BC': 1000,
 'divergence_loss': array(2.112265e-07, dtype=float32),
 'lambda_div': 0.1,
 'force_loss': array(1.6592154e-05, dtype=float32),
 'lambda_ff': 0.1}

In [19]:
key = ["BC_loss", "lambda_BC", 'Div_loss', "lambda_div", "FF_loss", "lambda_ff"]
value = [state['BC_loss'].item(), state['lambda_BC'], state['divergence_loss'].item(), state['lambda_div'], state['force_loss'].item(), state['lambda_ff']]

In [20]:
df_loss = pd.DataFrame.from_dict([dict(zip(key, value))])

In [21]:
df_loss

,BC_loss,lambda_BC,Div_loss,lambda_div,FF_loss,lambda_ff
0,0.040236,1000,2.112265e-07,0.1,0.000017,0.1


In [22]:
df_new = metric_df(B=B, b=b, B_potential=b_potential, iteration=int(iters))
df_new

,iter,C_vec,C_cs,1-En,1-Em,eps,eps_p,sig_J,L1,L2,curlB
0,0,0.071152,0.3354,-1.370883,-6.070672,0.718241,1.037789,0.558127,0.000165,0.000002,0.055361


In [23]:
df_new =pd.concat([df_new, df_loss], axis=1)

In [24]:


df = pd.concat([df, df_new], ignore_index=True)
print('metric: ', file_path)

metric:  results/simul3/run/fields_000000.nf2


In [25]:
df

,iter,C_vec,C_cs,1-En,1-Em,eps,eps_p,sig_J,L1,L2,curlB,BC_loss,lambda_BC,Div_loss,lambda_div,FF_loss,lambda_ff
0,-2,1.000000,1.000000,1.000000,1.000000,1.000000,1.444904,0.013080,2.064561e-03,0.002024,1.000000e+00,NaN,NaN,NaN,NaN,NaN,NaN
1,-1,0.864653,0.869255,0.430030,0.362425,0.692088,1.000000,0.853069,7.271669e-13,0.084496,2.762271e-07,NaN,NaN,NaN,NaN,NaN,NaN
2,0,0.071152,0.335400,-1.370883,-6.070672,0.718241,1.037789,0.558127,1.652433e-04,0.000002,5.536096e-02,0.040236,1000.0,2.112265e-07,0.1,0.000017,0.1


In [26]:
import os
import torch
import glob
from torchsummary import summary as summary

In [27]:
field_files = os.path.join(info['simul']['base_path'],'fields_*.nf2')
field_files = sorted(glob.glob(field_files))

In [28]:
file_file = field_files[0]

In [29]:
state = torch.load(file_file)

In [30]:
state['model']

BModel(
  (d_in): Linear(in_features=3, out_features=256, bias=True)
  (linear_layers): ModuleList(
    (0-7): 8 x Linear(in_features=256, out_features=256, bias=True)
  )
  (d_out): Linear(in_features=256, out_features=3, bias=True)
  (activation): Sine()
)

In [31]:
summary(state['model'], (3,))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Linear-1                  [-1, 256]           1,024
              Sine-2                  [-1, 256]               0
            Linear-3                  [-1, 256]          65,792
              Sine-4                  [-1, 256]               0
            Linear-5                  [-1, 256]          65,792
              Sine-6                  [-1, 256]               0
            Linear-7                  [-1, 256]          65,792
              Sine-8                  [-1, 256]               0
            Linear-9                  [-1, 256]          65,792
             Sine-10                  [-1, 256]               0
           Linear-11                  [-1, 256]          65,792
             Sine-12                  [-1, 256]               0
           Linear-13                  [-1, 256]          65,792
             Sine-14                  [